In [ ]:
%load_ext autoreload
%autoreload 2

# Sonecule: DiscreteCallbackPMS - DiscretePMS Sonecule using a Callback function for flexible mapping

This notebook introduces and demonstrates usage of the `DiscreteCallbackPMS` sonecule.
* The sonecule spawns an instance of a synth for each data point in a data set.
* The synth offers several parameters that can be set only at init time
* Specifically the parameters are:
  * amplitude
  * frequency
  * sharpness
  * spatial panning
  * attack time
  * duration of the event
  * release time
* Different from the `DiscretePMS`, where the mapping is specified by a mapping specification dictionary that is parsed column by column, this sonecule calls a callback function for each row, providing the row vector as an argument. 
* this allows highly flexible mappings and even things that are impossible with the non-callback Sonecule, such as suppressing sounds if certain conditions throughout channels apply.
* Definition of a callback function is a daunting task for non-programmers. Therefore the sonecule comes with a `create_callback_template` method, that delivers as output a function (string) that can be copied and pasted in a jupyter notebook cell (or your IDE), as starting point for own sonification designs using the given data set and given synth.

## Data Preparation

In [ ]:
# headers and imports for the demo
import sonecules as sn
from pya import Asig
import pyamapping as pam
import matplotlib.pyplot as plt

# setup for matplotlib 
plt.rcParams["figure.figsize"] = (8,3)
%matplotlib widget

# start sonecules (with default backend sc3nb, aka sc3)
sn.startup()
ctx = sn.gcc()  # get the context as ctx

In [ ]:
%run ../data/prepare-data.ipynb

For this sonecule we work with the building data set for the demonstrations. That is a data set that describes the hourly water and electricity consumption of an office building plus environmental features such as temperature, humidity, wind-speed, time of day, solar radiation. With roughly 4300 rows the data set contains roughly half a year of data.

In [ ]:
df = dataframes["building"]
df.plot(lw=0.2);

In [ ]:
sns.pairplot(df.iloc[2000:, 10:], height=0.8, plot_kws={"s": 3});

## Usage / Demo Snippets

Let's create a DiscreteCallbackPMS using a callback function for mapping.

First we need to import the sonecule

In [ ]:
from sonecules.scoreson import DiscreteCallbackPMS, mapcol

The following code cell shows all steps usually used to sonify the data, i.e., 
- creation of the sonecule
- cleaning the time line
- defining the callback function (here named `callback_fn`, but any name is fine)
- creating the sonification using the sonecules schedule function
- starting your sonification using the sonecule start function

For the example we start with one week =7*24 hourly measurments of the building data set

Note that the callback function receives the following arguments:
- r: the row vector from the data set
- cmi: the dictionary of all columns' minimum values (useful for the mapcol function)
- cma: the dictionary of all columns' maximum values (useful for the mapcol function)
- pp: the dictionary containing all parameters as keys and their default values as value
  - so if pp entries are not overwritten, the default will be used

For mapping any python based code can be used, including if/else statements and computations.
- a common case is to map values of a specific feature (e.g. column $k$) to the parameter
- to simplify this usecase, we offer the `mapcol(r, column_name, cmi, cma, p1, p2)` function:
  - it maps column `column_name` (string or integer)
  - from source range [ cmi[column_name], cma[column_name] ] 
  - to a target range [p1, p2], note that p1 < p2 is not necessary...
  - using a linear mapping function without clipping.
- Any nonlinear warping can happen as developers see fit, e.g. see mapping for 'amp' below

In [ ]:
# create the sonecule
scb = DiscreteCallbackPMS(df.iloc[:7*24, 10:])  # one week, omit the first 6 columns

# clear the timeline
ctx.timeline.reset()

# we use the mapcol helper function which maps the columns data 
# mapcol(data_row, feature, column_mins, column_maxs, target_min, target_max)

# define the callback function as needed/wanted, for example
def callback_fn(r, cmi, cma, pp):
    pp['onset']     = mapcol(r, 'humidity', cmi, cma, 0, 3)
    pp['freq']      = pam.midi_to_cps(mapcol(r, 'temperature', cmi, cma, 58, 82))
    pp['amp']       = pam.db_to_amp(mapcol(r, 'solar_radiation', cmi, cma, -40, -20)) 
    pp['pan']       = mapcol(r, 'hc_wb_electrical', cmi, cma, -1, 1)
    return pp # it returns the mapping dictionary

# sonify the data using the above callback function
scb.schedule(at=0, callback_fn=callback_fn)

# start the playback 
scb.start(rate=1);

- play with the definition of the callback_fn and execute again to explore the data
- to see available column names use

In [ ]:
scb.data.columns

- control the duration and data slice.
- Once you want to keep the callback function fixed there is no need to reexecute
- Here some other data sonified with the same callback function

In [ ]:
scb = DiscreteCallbackPMS(df.iloc[30*24:60*24, 9:])  # one month (omitting the first 30 days) of data in 3 seconds
ctx.timeline.reset()
scb.schedule(at=0, callback_fn=callback_fn).start();

If wanted we can have a sonification that specifies or uses more parameters of the synth.
If you use the default synth ("dcbpms"), its parameters are

In [ ]:
scb.syn.params

As of now synths do not come with given bounds, but it is useful to design and specify them and later sonecules can pick from a library of curated synth definitions with suitable bounds.
Let's define a more complete mapping

In [ ]:
scb = DiscreteCallbackPMS(df.iloc[::2, 9:])  # all data, every 2nd row (>6m) in 15 seconds

# here a mapping function that specifies all synth parameters 
def callback_fn(r, cmi, cma, pp):
    pp['onset']     = mapcol(r, 'humidity', cmi, cma, 0, 9)
    pp['freq']      = pam.midi_to_cps(mapcol(r, 'temperature', cmi, cma, 58, 82))
    pp['amp']       = pam.db_to_amp(mapcol(r, 'solar_radiation', cmi, cma, -30, -10)) 
    pp['pan']       = mapcol(r, 'hc_wb_electrical', cmi, cma, -1, 1)
    pp['sharp']     = mapcol(r, 'solar_radiation', cmi, cma, 1, 12)
    pp['vibfreq']   = pam.linlin(r['hc_wb_hot_water'], -0.5, 0.5, 3, 8)
    pp['vibir']     = 0.05
    pp['dur']       = 0.1
    pp['rel']       = pp['dur']
    return pp # it returns the mapping dictionary
ctx.timeline.reset()
scb.schedule(at=0.0, callback_fn=callback_fn).start();

- As this is all executed in real-time, you will probably experience limits of your system.
- Sonecules will issue late warnings if there are processing-based delays
- in such cases, a non-real-time rendering is always an option to compute a guaranteed correct sonification.

In [ ]:
# Render the current Timeline as asig
asig = ctx.render_asig()

The audio has been rendered to a file and then loaded into an pya.Asig, so that 
it can be plotted, played, etc.

In [ ]:
# plot the sonification audio
plt.figure()
asig[:, :2].plot(lw=0.25, offset=1)  # plot the first two channels

To play the asig via pya we first need to start the Aserver.
(jump over that in case not needed/wanted here)

In [ ]:
from pya import startup
s = startup()

In [ ]:
asig.play()

In [ ]:
ctx.stop()  # execute this in case an error occurs and synths keep on playing

Usually users prefer / wish to use their own synths. This can be done as follows:

In [ ]:
# create your synths - this depends on the backend 
# in this case we use sc3nb and provide a SuperCollider UGen Graph Function for a SynthDef
ctx.synths.add_synth_def("mysyn", r"""{ | out=0, freq=400, dur=0.1, amp=0.5, rq=0.1, pan=0 |
    var noise = WhiteNoise.ar(amp);
    var filt = BPF.ar(noise, freq, rq);
    var env = Line.kr(1, 0, dur, doneAction: 2);
    Out.ar(out, Pan2.ar(filt, pan, env));
}""")

# create sonecule
scb = DiscreteCallbackPMS(df.iloc[::2, 9:], "mysyn")  # all data, every 2nd row (>6m) in 15 seconds

# here a mapping function that specifies all synth parameters 
def callback_fn(r, cmi, cma, pp):
    pp['onset'] = mapcol(r, 'humidity', cmi, cma, 0, 4)
    pp['freq']  = pam.midi_to_cps(mapcol(r, 'temperature', cmi, cma, 58, 82))
    pp['amp']   = pam.dbamp(mapcol(r, 'solar_radiation', cmi, cma, -10, 10)) 
    pp['pan']   = mapcol(r, 'hc_wb_electrical', cmi, cma, -1, 1)
    pp['dur']   = 0.1
    pp['rq']    = 0.1
    return pp # it returns the mapping dictionary

ctx.timeline.reset()

scb.schedule(at=0.0, callback_fn=callback_fn).start();

Now let the sonecule propose a mapping as starting point for own experiments.

- Calling `create_callback_template()` 
- composes and then prints the python code string.
- The output can be copied into a notebook cell and adapted as needed.
- On execution it defines the callback function cbfn().
- The sonification can be rendered as you go until satisfactory.

In [ ]:
scb = DiscreteCallbackPMS(df.iloc[:7*24, 8:], "mysyn")  # first week but no weekday features
fnstr = scb.create_callback_template(auto_assign=True, duration=8)

The printout shows python code 
- that you can simply copy and paste into a new code cell
- ATTENTION: note that the code might not be delivering useful sound as such
  - for instance: 'onset' may be mapped from a feature in which data don't spread nicely...
  - so best is to check the mapping, adjust so that it makes sense 
- modify the cell as you see fit, e.g. remove rows, or set the right hand side to a constant as preferred
- once satisfied, you can execute the function.
- Note that the code line with the schedule is commented out: the variable name of the sonecule is assumed to be scb, so this has to be manually adjusted if it's different


In [ ]:
def cbfn(r, cmi, cma, pp):
    # columns are:'hour' 'am_pm' 'temperature' 'humidity' 
    # 'solar_radiation' 'wind_speed' 'hc_wb_electrical' 'hc_wb_cold_water' 
    # 'hc_wb_hot_water' 
    pp['onset']    	 = mapcol(r, 'temperature', cmi, cma, 0.00, 8.00)
    pp['freq']     	 = mapcol(r, 'am_pm', cmi, cma, 300.00, 600.00)
    pp['dur']      	 = mapcol(r, 'temperature', cmi, cma, 0.08, 0.15)
    pp['amp']      	 = mapcol(r, 'humidity', cmi, cma, 0.38, 0.75)
    pp['rq']       	 = mapcol(r, 'solar_radiation', cmi, cma, 0.01, 0.05)
    pp['pan']      	 = mapcol(r, 'wind_speed', cmi, cma, 0.00, 0.00)
    return pp
# create sonification e.g. by using
sn.gcc().timeline.reset()
scb.schedule(at=0, callback_fn=cbfn).start(rate=1);

Let's create another sonification for the mesonic default synth "s1"

In [ ]:
scb = DiscreteCallbackPMS(df.iloc[:60*24, 10:], "s1")  # for 2 months data
fnstr = scb.create_callback_template(auto_assign=True, duration=8)

In [ ]:
# this cell just is copied the output of the previous cell for a test...
def cbfn(r, cmi, cma, pp):
    # columns are:'temperature' 'humidity' 'solar_radiation' 'wind_speed' 
    # 'hc_wb_electrical' 'hc_wb_cold_water' 'hc_wb_hot_water' 
    pp['onset']    	 = mapcol(r, 'temperature', cmi, cma, 0.00, 8.00)
    pp['freq']     	 = mapcol(r, 'humidity', cmi, cma, 300.00, 600.00)
    pp['amp']      	 = mapcol(r, 'solar_radiation', cmi, cma, 0.23, 0.45)
    pp['num']      	 = mapcol(r, 'wind_speed', cmi, cma, 3.00, 6.00)
    pp['pan']      	 = mapcol(r, 'hc_wb_electrical', cmi, cma, 0.00, 0.00)
    pp['dur']      	 = mapcol(r, 'hc_wb_cold_water', cmi, cma, 0.30, 0.60)
    pp['att']      	 = mapcol(r, 'temperature', cmi, cma, 0.01, 0.01)
    pp['curve']    	 = mapcol(r, 'humidity', cmi, cma, -1.50, -3.00)
    return pp
# create sonification e.g. by using
sn.gcc().timeline.reset()
scb.schedule(at=0, callback_fn=cbfn).start(rate=1);

And finally here an example of an hand-crafted mapping, from modifying the above code, i.e.
- refining the mapping target ranges
- applying some useful non-linearities, e.g. mapping pitch to MIDI, then converting to freq
- setting some values as constants (here att and curve)
- exchaning some features e.g. cold water should control the sharpness (num) and wind speed the panning

In [ ]:
# this cell just is copied the output of the previous cell for a test...
def cbfn(r, cmi, cma, pp):
    # columns are:'temperature' 'humidity' 'solar_radiation' 'wind_speed' 
    # 'hc_wb_electrical' 'hc_wb_cold_water' 'hc_wb_hot_water' 
    pp['onset']    	 = mapcol(r, 'temperature', cmi, cma, 0.00, 5.00)
    pitch            = mapcol(r, 'humidity', cmi, cma, 40, 100)
    pp['freq']     	 = pam.midi_to_cps(pitch)
    pp['amp']      	 = pam.db_to_amp(mapcol(r, 'solar_radiation', cmi, cma, -20, 0))
    pp['num']      	 = mapcol(r, 'hc_wb_electrical', cmi, cma, 3, 10)
    pp['pan']      	 = mapcol(r, 'wind_speed', cmi, cma, -1, 1)
    pp['dur']      	 = mapcol(r, 'hc_wb_cold_water', cmi, cma, 0.05, 0.20)
    pp['att']      	 = 0.000
    pp['curve']    	 = -2
    return pp
# create sonification e.g. by using
scb.reschedule(at=0, callback_fn=cbfn).start(rate=1);

In [ ]:
ctx.close()  # close the mesonic context, exits backend gracefully